In [5]:
import sys
sys.path.append('..')
from src.TemporalEntityRecognizer import TemporalEntityRecognizer

language = "en"
entity_type = "TEMPORAL"
input_text = "the artefact was medieval or circa 1250-1275 or maybe post medieval? Underlying the modern made ground on the site was a layer covering the entire shaft area. This has been dated to c.1480-1800/1900 and interpreted as a post-medieval cultivation soil. Historic mapping illustrates that the site remained undeveloped through the post medieval period until the mid-19th century, when urban development around the site accelerated and construction of railways in this part of London began. On Gascoigne's 1703 map the site was open ground, the later maps of Rocque in 1746 and Horwood in 1799 show the area was in use as fields and Stanford's map of 1862 depicts the area surrounding Eleanor Street comprising of market gardens. These are all consistent with the archaeological evidence. Underlying the layer were natural terrace gravels. The archaeological fieldwork has demonstrated that remains relating to the Prehistoric, Roman or medieval period have not survived to the modern era, if they were once present on site."
output_format = "html"
ner = TemporalEntityRecognizer(language, entity_type)
entities = ner.get_entities(input_text)
formatted = TemporalEntityRecognizer.format_entities(input_text, entities, output_format)
print(formatted)

None
